### Emergency Detection

In [ ]:
from detection import detect_emergency, prepare_data_for_detection
from base.minio_utils import load_model_from_minio
from base.influx_utils import fetch_all_sensor_data

In [ ]:
sensor_data = fetch_all_sensor_data(
    start_hours=24,
    interval_hours=24
)
prepared_df = prepare_data_for_detection(sensor_data)
prepared_df.tail(5)

In [ ]:
room_stats = load_model_from_minio("occupancy")
room_stats

In [ ]:
detect_emergency()